In [ ]:
!pip install transformers==4.10.0
!pip install bert-extractive-summarizer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch

tokenizer = AutoTokenizer.from_pretrained('t5-base')
model = AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True)
device = torch.device("cpu")
model = model.to(device)

In [ ]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')

from textblob import Word

import random
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)

import torch
# from summarizer import Summarizer,TransformerSummarizer
import warnings
warnings.filterwarnings("ignore")


# Read csv and select required cols
df = pd.read_csv("/content/drive/MyDrive/FND using Unsupervised Learning /data/gossipcop/news_article_dataset.csv")
cols_to_select = ["id", "text","type"]
df = df[cols_to_select]

def preprocess(txt):
    try:
        txt = txt.decode('utf-8').lower()
    except:
        try:
            txt = txt.encode('utf-8').decode('utf-8').lower()
        except:
            print("TXT: "+str(txt))
            return txt
    txt = re.sub(r'\s+',' ',txt)
    txt = re.sub(r"\\n", " ",txt)
    txt = re.sub(r'@[\w\-]+','',txt)
    txt = re.sub(r'https?:\/\/\S+','',txt)
    txt = re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)",'',txt)
    txt = re.sub(r'{link}', '', txt)
    txt = re.sub(r"\[video\]", '', txt)
    txt = re.sub(r'&[a-z]+;', '', txt)
    txt = re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', txt)
    txt = re.sub(r"[^\w\d'\s]+",'',txt)
    return txt


stop_words = stopwords.words('english')
def get_tokens(tweet):
    tweet = preprocess(tweet)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(tweet)
      
    # Think whether to include this or not?
    #words = [Word(t).lemmatize() for t in tokens]
    #words = [w for w in words if not w in stop_words]
    
    words = ' '.join(tokens)
    return words

"""
def summarizer(body):
    GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
    summarized_tokens = GPT2_model(body)
    summary = ''.join(summarized_tokens)
    return summary
"""



def summarizer(body):
    inputs = tokenizer.encode("summarize: " + body,return_tensors='pt',max_length=512,truncation=True).to(device)
    summary_ids = model.generate(inputs, max_length=150, min_length=80, length_penalty=5., num_beams=2).to(device)
    summary = tokenizer.decode(summary_ids[0],skip_special_tokens=True)
    # print(summary)
    return summary


drop_rows = []
for i in range(df.shape[0]):
    try:    
        drop_rows.append(not np.isnan(df['text'].loc[i]))
    except:
        drop_rows.append(True)

df = df[drop_rows]

#df['text'] = df['text'].apply(lambda x: summarizer(x))
df['text'] = df['text'].apply(lambda x: get_tokens(x))

df.reset_index(drop=True)

max_len = 0
for i in range(df.shape[0]):
    tweet = df['text'].iloc[i]
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(tweet)
    max_len = max(max_len,len(tokens))

print("MAX LEN: "+str(max_len))

# /content/drive/MyDrive/FND using Unsupervised Learning /data/gossipcop/news_article_dataset.csv
df.to_csv("/content/drive/MyDrive/FND using Unsupervised Learning /data/gossipcop/processed_news_article_dataset.csv")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
